# **20230619**

## 17장 딥러닝을 이용한 자연어 처리


- 음성 -> 텍스트 : STT(speech to text)
- 텍스트 -> 음성 : TTS(text to speech)

### 01 텍스트의 토큰화

- keras가 제공하는 text 모듈의 text_to_word_sequence() 함수를 사용하면 문장을 단어 단위로 쉽게 나눌 수 있다.

In [34]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

text= '해보지 않으면 해낼 수 없다.'

result = text_to_word_sequence(text)
print("\n원문:\n", text)
print("\n토큰화:\n", result)


원문:
 해보지 않으면 해낼 수 없다.

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


In [35]:
# Tokenizer class 임포트
from tensorflow.keras.preprocessing.text import Tokenizer

In [36]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.'
        ]

In [37]:
token = Tokenizer() # 클래스 생성
token.fit_on_texts(docs)

print('단어카운트:\n',token.word_counts)

단어카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [38]:
print('문장 카운트:',token.document_count)

문장 카운트: 3


In [39]:
print('각 단어가 몇개의 문장에 포함되어 있는가\n',token.word_docs)

각 단어가 몇개의 문장에 포함되어 있는가
 defaultdict(<class 'int'>, {'나누어': 1, '텍스트의': 2, '토큰화합니다': 1, '단어를': 1, '각': 1, '먼저': 1, '토큰화해야': 1, '단어로': 1, '인식됩니다': 1, '딥러닝에서': 2, '결과는': 1, '토큰화한': 1, '수': 1, '있습니다': 1, '사용할': 1})


In [40]:
print('각 단어에 매겨진 인덱스 값 : \n',token.word_index)

# => Bag of Words
# 단어 임베딩 : 문자 -> 의미있는 숫자(=벡터)

각 단어에 매겨진 인덱스 값 : 
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


### 02 단어의 원-핫 인코딩

In [41]:
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'

token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [42]:
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [43]:
from tensorflow.keras.utils import to_categorical

# 인덱스 수에 하나를 추가해서 원-핫 인코딩 배열 만들기
word_size = len(token.word_index) + 1
x = to_categorical(x, num_classes=word_size)

print(x)
# [(0인덱스)  오랫동안  꿈꾸는  이는  그  꿈을  닮아간다] 
# 6차원 공간의 벡터 하나 =벡터화됨

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


### 03 단어 임베딩(word embedding)
- 단어사전 = 말뭉치


- 원핫 인코딩은 공간낭비가 큼 

- 단어 임베딩은 주어진 배열을 정해진 길이로 압축시킨다. => 차원축소(주성분추출)



In [44]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(16,4))
# 입력될 총 단어 수:16, 임베딩 후 출력되는 벡터 크기 : 4
# 단어를매번 얼마나 입력할 지 추가로 지정 가능 : input_length = 

### 04 텍스트를 읽고 긍정, 부정 예측하기

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

from numpy import array

In [46]:
docs = ['너무 재밌네요','최고예요','참 잘 만든 영화예요'
        ,'추천하고 싶은 영화입니다','한번 더 보고싶네요'
        ,'글쎄요','별로예요','생각보다 지루하네요'
        ,'연기가 어색해요','재미없어요']

In [47]:
# 긍정리뷰 = 1, 부정리뷰 = 0으로 클래스 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

In [48]:
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [49]:
# 순서파악용 안해도 상관없음
x = token.texts_to_sequences(docs)
print('리뷰 텍스트, 토큰화 결과:\n',x)

리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [65]:
len_list = []
for i in range(len(token.texts_to_sequences(docs))):

    len_list.append(len(token.texts_to_sequences(docs)[i]))

max(len_list)

4

In [50]:
padded_x = pad_sequences(x, 4)
print('패딩 결과:\n',padded_x)

패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [51]:
# 임베딩에 입력될  단어의 수 지정
word_size = len(token.word_index)+1

# 단어 임베딩을 포함해 딥러닝 모델을 만들고 결과를 출력

model = Sequential()
# word_size : 단어20개 -> 8차원으로 축소
# input_length=4 : 4개 단어씩 입력 처리
model.add(Embedding(word_size, 8 , input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))   # return 0~1
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 4, 8)              168       
                                                                 
 flatten_2 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(padded_x, classes, epochs=20)
print('Accuracy: %.4f'%(model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 1s 961ms/step - loss: 0.6909 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6886 - accuracy: 0.6000
Epoch 3/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6862 - accuracy: 0.7000
Epoch 4/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6839 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6816 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6792 - accuracy: 0.9000
Epoch 7/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6769 - accuracy: 0.9000
Epoch 8/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6746 - accuracy: 0.9000
Epoch 9/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6722 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6699 - accuracy: 0.9000
Epoch 11/20
1/1 [====================

1/1 [==============================] - 0s 209ms/step - loss: 0.6438 - accuracy: 0.9000
Accuracy: 0.9000
